In [1]:
#lib
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd


In [2]:
doc =  '../Data/Binance_1m_BTCUSDT_2020-01-01_2025-07-07.csv'
name = 'CG_BTC_H_future_orderbookaggregated.csv'
csv = pd.read_csv(doc)
csv


,Open time,Open,High,Low,Close,Volume,Close time,Quote asset volume,Number of trades,Taker buy base asset volume,Taker buy quote asset volume,Ignore
0,2020-01-01 00:00:00+00:00,7189.43,7190.52,7177.00,7182.44,246.092,2020-01-01 00:00:59.999000+00:00,1.767430e+06,336,46.630,3.348132e+05,0
1,2020-01-01 00:01:00+00:00,7182.43,7182.44,7178.75,7179.01,70.909,2020-01-01 00:01:59.999000+00:00,5.091458e+05,140,32.597,2.340633e+05,0
2,2020-01-01 00:02:00+00:00,7179.01,7179.01,7175.25,7177.93,99.420,2020-01-01 00:02:59.999000+00:00,7.135396e+05,148,16.311,1.170669e+05,0
3,2020-01-01 00:03:00+00:00,7177.77,7182.60,7177.00,7181.11,69.330,2020-01-01 00:03:59.999000+00:00,4.977934e+05,104,43.723,3.139200e+05,0
4,2020-01-01 00:04:00+00:00,7179.10,7179.10,7172.94,7175.25,97.368,2020-01-01 00:04:59.999000+00:00,6.986274e+05,193,36.616,2.627347e+05,0
...,...,...,...,...,...,...,...,...,...,...,...,...
2900995,2025-07-07 13:55:00+00:00,108250.00,108259.90,108203.40,108249.90,108.935,2025-07-07 13:55:59.999000+00:00,1.178913e+07,1847,61.500,6.655528e+06,0
2900996,2025-07-07 13:56:00+00:00,108249.90,108343.40,108247.20,108302.50,146.226,2025-07-07 13:56:59.999000+00:00,1.583528e+07,2995,98.824,1.070122e+07,0
2900997,2025-07-07 13:57:00+00:00,108302.40,108400.00,108294.40,108400.00,277.763,2025-07-07 13:57:59.999000+00:00,3.009795e+07,3667,205.719,2.229246e+07,0
2900998,2025-07-07 13:58:00+00:00,108399.90,108537.90,108399.90,108503.70,519.664,2025-07-07 13:58:59.999000+00:00,5.637026e+07,6750,354.391,3.844072e+07,0


In [ ]:
#timestamp to datetime 
#csv['datetime'] = pd.to_datetime(csv['start_time'],unit = "ms") # coinglass
csv['datetime'] = pd.to_datetime(csv['Open time']) # binance

if f'Unnamed: 0' in csv.columns:
   csv.drop(columns= 'Unnamed: 0',inplace= True)

csv


,Open time,Open,High,Low,Close,Volume,Close time,Quote asset volume,Number of trades,Taker buy base asset volume,Taker buy quote asset volume,Ignore,datetime
0,2020-01-01 00:00:00+00:00,7189.43,7190.52,7177.00,7182.44,246.092,2020-01-01 00:00:59.999000+00:00,1.767430e+06,336,46.630,3.348132e+05,0,2020-01-01 00:00:00+00:00
1,2020-01-01 00:01:00+00:00,7182.43,7182.44,7178.75,7179.01,70.909,2020-01-01 00:01:59.999000+00:00,5.091458e+05,140,32.597,2.340633e+05,0,2020-01-01 00:01:00+00:00
2,2020-01-01 00:02:00+00:00,7179.01,7179.01,7175.25,7177.93,99.420,2020-01-01 00:02:59.999000+00:00,7.135396e+05,148,16.311,1.170669e+05,0,2020-01-01 00:02:00+00:00
3,2020-01-01 00:03:00+00:00,7177.77,7182.60,7177.00,7181.11,69.330,2020-01-01 00:03:59.999000+00:00,4.977934e+05,104,43.723,3.139200e+05,0,2020-01-01 00:03:00+00:00
4,2020-01-01 00:04:00+00:00,7179.10,7179.10,7172.94,7175.25,97.368,2020-01-01 00:04:59.999000+00:00,6.986274e+05,193,36.616,2.627347e+05,0,2020-01-01 00:04:00+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2900995,2025-07-07 13:55:00+00:00,108250.00,108259.90,108203.40,108249.90,108.935,2025-07-07 13:55:59.999000+00:00,1.178913e+07,1847,61.500,6.655528e+06,0,2025-07-07 13:55:00+00:00
2900996,2025-07-07 13:56:00+00:00,108249.90,108343.40,108247.20,108302.50,146.226,2025-07-07 13:56:59.999000+00:00,1.583528e+07,2995,98.824,1.070122e+07,0,2025-07-07 13:56:00+00:00
2900997,2025-07-07 13:57:00+00:00,108302.40,108400.00,108294.40,108400.00,277.763,2025-07-07 13:57:59.999000+00:00,3.009795e+07,3667,205.719,2.229246e+07,0,2025-07-07 13:57:00+00:00
2900998,2025-07-07 13:58:00+00:00,108399.90,108537.90,108399.90,108503.70,519.664,2025-07-07 13:58:59.999000+00:00,5.637026e+07,6750,354.391,3.844072e+07,0,2025-07-07 13:58:00+00:00


In [15]:
min_date = csv['datetime'].min()
max_date = csv['datetime'].max()
full_range = pd.date_range(start=min_date, end=max_date, freq='1min') #1H , 1D
csv.set_index('datetime')
csv.reindex(full_range)
csv.to_csv(doc)

In [14]:

full_range = pd.date_range(start=min_date, end=max_date, freq='1min') #1H , 1D
csv.set_index('Date')
csv.reindex(full_range)
csv.index.name = 'Date'
csv

KeyError: "None of ['Date'] are in the columns"

In [16]:
missing = csv.isnull().sum()
print(f'missing_places{missing}')



missing_placesOpen time                       0
Open                            0
High                            0
Low                             0
Close                           0
Volume                          0
Close time                      0
Quote asset volume              0
Number of trades                0
Taker buy base asset volume     0
Taker buy quote asset volume    0
Ignore                          0
datetime                        0
dtype: int64
